## Tests to see if classifier works

In [ ]:
import sys
import os
from pathlib import Path
DIR = Path(".").absolute()
sys.path.append(str(DIR))
os.chdir(str(DIR.parent)) # make notebook assume its in parent dir

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import json
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [3]:
from dask.distributed import Client

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://172.31.40.124/3037/1 Dashboard: http://172.31.40.124:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


`processes=False` is a large performance increase because threads have much faster communication when a Dask Distributed client is present. At each step, the model is copied to each worker. That's almost instant with threads because the same memory bank is shared; that's not true with processes.

`processes=False` should still be used with GPUs. There, the model/gradient calculation will live on each GPU.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from adadamp import DaskClassifier

In [6]:
# model from https://github.com/pytorch/examples/blob/master/mnist/main.py
from model import Net
client.upload_file("notebooks/model.py")

In [7]:
def get_model_weights(model):
    s = 0
    for param in model.parameters():
        s += torch.abs(torch.sum(param)).item()
    return s

In [8]:
from torch.utils.data import Dataset
from dask.distributed import get_client

def run(
    model: nn.Module,
    train_set: Dataset,
    test_set: Dataset,
    max_epochs: int = 5,
):
    client = get_client()
    hist = []
    epochs = 0
    for epoch in range(max_epochs):
        # train
        print(f"Epoch {epoch}...", end=" ")
        pre_weights = get_model_weights(model.module_)
        model.partial_fit(train_set)
        print("done")
        
        # ensure update
        assert pre_weights != get_model_weights(model.module_), "ERROR: Model weights not changed after partial fit"
        
        # test model
        # temporarily using train set for testing
        model.score(train_set) # records info in model.meta_
        
        assert model.meta_['n_updates'] == model.meta_['n_weight_changes']
        
        datum = {"epoch": epoch + 1, **model.meta_}
        print(datum)
        hist.append(datum)
    return hist, model.get_params()

In [9]:
# transforms
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# data
train_set = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST('./data', train=False, transform=transform)

In [10]:
# model
model = DaskClassifier(
    module=Net,
    weight_decay=1e-5,
    loss=nn.CrossEntropyLoss,
    optimizer=optim.Adagrad,
    batch_size=1024,
    device="cpu" if not torch.cuda.is_available() else "cuda:0"
)

In [ ]:
args = (model, train_set, test_set)
kwargs = dict(max_epochs=20)
hist, params = run(*args, **kwargs)

Epoch 0... done
{'epoch': 1, 'n_updates': 59, 'n_data': 60416, 'n_weight_changes': 59, 'score__calls': 1, 'partial_fit__calls': 1, 'n_workers': 32, 'partial_fit__time': 28.44567084312439, 'partial_fit__batch_size': 1024, 'weight_aggregate': 1187.5236362293363, 'score__acc': 0.9106833338737488, 'score__loss': 0.3267969517389933, 'score__time': 9.503589153289795}
Epoch 1... done
{'epoch': 2, 'n_updates': 118, 'n_data': 120832, 'n_weight_changes': 118, 'score__calls': 2, 'partial_fit__calls': 2, 'n_workers': 32, 'partial_fit__time': 28.127146005630493, 'partial_fit__batch_size': 1024, 'weight_aggregate': 1243.6873664408922, 'score__acc': 0.9174833297729492, 'score__loss': 0.33157633209228515, 'score__time': 9.445111274719238}
Epoch 2... done
{'epoch': 3, 'n_updates': 177, 'n_data': 181248, 'n_weight_changes': 177, 'score__calls': 3, 'partial_fit__calls': 3, 'n_workers': 32, 'partial_fit__time': 28.128308296203613, 'partial_fit__batch_size': 1024, 'weight_aggregate': 1262.7476951479912, 's